In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI SFT Gemini Migration Recipe


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/sft_gemini_migration_recipe.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Ftuning%2Fsft_gemini_migration_recipe.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/tuning/sft_gemini_migration_recipe.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/sft_gemini_migration_recipe.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/bigquery/v1/32px.svg" alt="BigQuery Studio logo"><br> Open in BigQuery Studio
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/sft_gemini_migration_recipe.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/sft_gemini_migration_recipe.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/sft_gemini_migration_recipe.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/sft_gemini_migration_recipe.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/sft_gemini_migration_recipe.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/sft_gemini_migration_recipe.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| Author(s) |
| --- |
| [Jeremy Hao](https://github.com/zh272)
| [Erwin Huizenga](https://github.com/erwinh85)

## Overview

This colab provides migration recipe for Gemini managed tuning on the following migration pairs:

- `gemini-1.5-flash-002` -> `gemini-2.5-flash`

Gemini tuning supports the following hyperparameters:

- **Epochs**: The number of complete passes the model makes over the entire training dataset during training.
- **Adapter size**: The Adapter size to use for the tuning job. The adapter size influences the number of trainable parameters for the tuning job. A larger adapter size implies that the model can learn more complex tasks, but it requires a larger training dataset and longer training times.
- **Learning Rate Multiplier**: A multiplier to apply to the recommended learning rate. You can increase the value to converge faster, or decrease the value to avoid overfitting.

The values for these hyperparameters impacts the quality of tuned models. Because of model architecture change and tuning infra changes, we do NOT recommend to apply the same hyperparameters from legacy Gemini models to the latest Gemini models.




Note that the provided migration recipe below is for those tuning jobs whose hyper-parameters were set EXPLICITLY on legacy models. In cases where the tuning job to be migrated used default hyper-parameters (i.e. you did not specify hyperparameters explicitly), you can rely on the API or SDK to populate new default values for you on new models.

## Get started

### Install Google Gen AI SDK and other required packages


In [ ]:
%pip install --upgrade --quiet google-genai

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "global")

from google import genai

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Import libraries

In [ ]:
from IPython.display import Markdown, display

In [ ]:
# @title library
import math
import json
from google.genai import types

def get_tuning_job_hparams(tuning_job):
  job_specs = {}
  job_specs['state'] = tuning_job.state
  job_specs['base_model'] = tuning_job.base_model
  job_specs['tuned_model_display_name'] = tuning_job.tuned_model_display_name

  supervised_tuning_spec = tuning_job.supervised_tuning_spec
  job_specs['training_dataset_uri'] = supervised_tuning_spec.training_dataset_uri
  job_specs['validation_dataset_uri'] = supervised_tuning_spec.validation_dataset_uri

  hparams = supervised_tuning_spec.hyper_parameters
  job_specs['adapter_size'] = hparams.adapter_size
  job_specs['epoch_count'] = hparams.epoch_count
  job_specs['learning_rate_multiplier'] = hparams.learning_rate_multiplier

  sft_data_stats = tuning_job.tuning_data_stats.supervised_tuning_data_stats
  job_specs['sft_total_tokens'] = sft_data_stats.total_billable_token_count
  job_specs['sft_num_examples'] = sft_data_stats.tuning_dataset_example_count
  job_specs['sft_avg_seq_len'] = job_specs['sft_total_tokens'] / job_specs['sft_num_examples']
  # WARNING: API does not give max_seq_len, so using max_input_len+max_output_len as a surrogate, but this may be inaccurate.
  job_specs['sft_max_seq_len'] = sft_data_stats.user_input_token_distribution.max + sft_data_stats.user_output_token_distribution.max

  return job_specs


def epoch_1_5_flash_to_2_5_flash_short_context(old_epochs: int):
  new_epochs = old_epochs
  if 10< old_epochs < 80:
    new_epochs = math.ceil(10 + 6 * (old_epochs-10) / 70)
  elif old_epochs >= 80:
    new_epochs = 0.2 * old_epochs
  return math.ceil(new_epochs)

def lrm_1_5_flash_to_2_5_flash(old_lrm: float):
  max_lrm = 10
  new_lrm = math.log(1+old_lrm)/math.log(2)
  new_lrm = min(max_lrm, new_lrm)
  return round(new_lrm, 1)

def gemini_1_5_flash_to_gemini_2_5_flash(old_specs: dict):
  if old_specs.get('base_model') != 'gemini-1.5-flash-002':
    return None
  new_specs = {
      'base_model': 'gemini-2.5-flash',
      'tuned_model_display_name': old_specs['tuned_model_display_name'] + '_migrated_gemini_2_5_flash',
      'training_dataset_uri': old_specs['training_dataset_uri'],
      'validation_dataset_uri': old_specs['validation_dataset_uri'],
  }
  if 'adapter_size' in old_specs:
    new_specs['adapter_size'] = old_specs['adapter_size']

  if 'epoch_count' in old_specs:
    old_epochs = old_specs['epoch_count']
    if old_specs['sft_avg_seq_len'] <= 700 and old_specs['sft_max_seq_len'] <= 8192 and old_specs['sft_num_examples'] < 10000:
      new_epochs = epoch_1_5_flash_to_2_5_flash_short_context(old_epochs)
    elif old_specs['sft_avg_seq_len'] <= 3000 and old_specs['sft_max_seq_len'] > 8192:
      new_epochs = epoch_1_5_flash_to_2_5_flash_short_context(old_epochs)
    else:
      new_epochs = old_epochs
    new_specs['epoch_count'] = new_epochs

  if 'learning_rate_multiplier' in old_specs:
    new_specs['learning_rate_multiplier'] = lrm_1_5_flash_to_2_5_flash(
        old_specs['learning_rate_multiplier']
    )
  return new_specs


## Example

In [ ]:
# @title Load legacy tuning job

legacy_tuning_job_number = "[your-tuning-job-number]"  # @param {type:"string"}
if not legacy_tuning_job_number or legacy_tuning_job_number == "[your-tuning-job-number]":
  raise Exception("Please provide a tuning job number")

legacy_tuning_job_name = f'projects/{PROJECT_NUMBER}/locations/{LOCATION}/tuningJobs/{legacy_tuning_job_number}'
print('Double check tuning job name:', legacy_tuning_job_name)
legacy_tuning_job = client.tunings.get(name=legacy_tuning_job_name)


In [ ]:
# @title gemini-1.5-flash-002 --> gemini-2.5-flash

legacy_job_specs = get_tuning_job_hparams(legacy_tuning_job)
new_job_specs = gemini_1_5_flash_to_gemini_2_5_flash(legacy_job_specs)
print('[1.5-flash tuning hparams]:')
print(json.dumps(legacy_job_specs, indent=2, default=str))
print('[2.5-flash tuning hparams]:')
print(json.dumps(new_job_specs, indent=2, default=str))

In [ ]:
# @title Create tuning job with new base_model

tuning_job = client.tunings.tune(
    base_model=new_job_specs['base_model'],
    training_dataset=types.TuningDataset(
        gcs_uri=new_job_specs['training_dataset_uri'],
    ),
    config=types.CreateTuningJobConfig(
        epoch_count=new_job_specs['epoch_count'],
        adapter_size=new_job_specs['adapter_size'],
        learning_rate_multiplier=new_job_specs['learning_rate_multiplier'],
        tuned_model_display_name=new_job_specs['tuned_model_display_name'],
        validation_dataset=types.TuningValidationDataset(
            gcs_uri=new_job_specs['validation_dataset_uri'],
        ),
    ),
)
print(tuning_job)